In [86]:
import pandas as pd

In [87]:
ratings = pd.read_csv(r'C:\Users\alfem\Downloads\archive\ratings.csv', usecols=['userId', 'movieId', 'rating'])
keywords = pd.read_csv(r'C:\Users\alfem\Downloads\archive\keywords.csv')
movies = pd.read_csv(r'C:\Users\alfem\Downloads\archive\movies_metadata.csv', usecols=['genres', 'id', 'original_title', 'popularity', 'release_date', 'vote_average', 'vote_count', 'original_language'])

C:\Users\alfem\AppData\Local\Temp\ipykernel_3332\2703897529.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(r'C:\Users\alfem\Downloads\archive\movies_metadata.csv', usecols=['genres', 'id', 'original_title', 'popularity', 'release_date', 'vote_average', 'vote_count', 'original_language'])


In [88]:
print(ratings.shape)
print(keywords.shape)
print(movies.shape)

(26024289, 3)
(46419, 2)
(45466, 8)


In [89]:
import pandas as pd

# Ensure consistent data types for movie IDs
ratings['movieId'] = ratings['movieId'].astype(str)
movies['id'] = movies['id'].astype(str)
keywords['id'] = keywords['id'].astype(str)
# Step 1: Check for movie IDs in ratings that do not appear in movies dataset
rated_movies_not_in_metadata = set(ratings['movieId']) - set(movies['id'])
print(f"Rated movie IDs not in metadata: {len(rated_movies_not_in_metadata)}")

# Step 2: Check for movie IDs in movies dataset that do not appear in ratings
movies_not_rated = set(movies['id']) - set(ratings['movieId'])
print(f"Movies in metadata but not rated: {len(movies_not_rated)}")

# Step 3: Filter movies
# Convert release_date to datetime and filter movies released after 1990
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')
filtered_movies = movies[movies['release_date'].dt.year >= 1990]

# Filter movies with sufficient votes and high average rating
min_votes = 150  # Minimum number of votes
min_vote_average = 6.0  # Minimum average rating
filtered_movies = filtered_movies[
    (filtered_movies['vote_count'] >= min_votes) & 
    (filtered_movies['vote_average'] >= min_vote_average)
]

# Step 4: Filter by language (keep only English, French, Italian movies)
allowed_languages = ['en', 'fr', 'it']  # Languages to keep
filtered_movies = filtered_movies[filtered_movies['original_language'].isin(allowed_languages)]

# Step 5: Ensure consistent filtering across datasets
# Keep only movies present in the filtered movies DataFrame
filtered_movie_ids = set(filtered_movies['id'])

# Filter the keywords DataFrame to include only movies in filtered_movie_ids
filtered_keywords = keywords[keywords['id'].isin(filtered_movie_ids)]

# Filter the ratings DataFrame to include only movies in filtered_movie_ids
filtered_ratings = ratings[ratings['movieId'].isin(filtered_movie_ids)]

# Step 6: Remove movies from filtered_movies that don't have corresponding ratings
rated_movie_ids = set(filtered_ratings['movieId'])
filtered_movies = filtered_movies[filtered_movies['id'].isin(rated_movie_ids)]

# Step 7: Remove keywords for movies that no longer exist in filtered_movies
filtered_movie_ids = set(filtered_movies['id'])  # Update the valid movie IDs
filtered_keywords = filtered_keywords[filtered_keywords['id'].isin(filtered_movie_ids)]

# Step 8: Remove users who rated fewer than 10 movies
# Count ratings for each user
user_rating_counts = filtered_ratings.groupby('userId').size()

# Filter users who have rated at least 10 movies
valid_users = user_rating_counts[user_rating_counts >= 30].index

# Filter the ratings DataFrame to include only valid users
filtered_ratings = filtered_ratings[filtered_ratings['userId'].isin(valid_users)]

# Step 9: Ensure filtered_movies only includes movies present in filtered_ratings
rated_movie_ids = set(filtered_ratings['movieId'])
filtered_movies = filtered_movies[filtered_movies['id'].isin(rated_movie_ids)]

# Step 10: Ensure filtered_keywords only includes movies present in filtered_movies
filtered_movie_ids = set(filtered_movies['id'])
filtered_keywords = filtered_keywords[filtered_keywords['id'].isin(filtered_movie_ids)]

# Output statistics for consistency check
print(f"Filtered movies count: {filtered_movies.shape[0]}")
print(f"Filtered keywords count: {filtered_keywords.shape[0]}")
print(f"Filtered ratings count: {filtered_ratings.shape[0]}")   
print(f"Filtered users count: {filtered_ratings['userId'].nunique()}")



Rated movie IDs not in metadata: 37550
Movies in metadata but not rated: 37871
Filtered movies count: 843
Filtered keywords count: 843
Filtered ratings count: 1652977
Filtered users count: 28463


In [90]:
print(filtered_movies.shape)
print(filtered_ratings.shape)
print(filtered_keywords.shape)

(843, 8)
(1652977, 3)
(843, 2)


In [91]:
import ast

def extract_keywords_or_genres(data):
    if isinstance(data, str):
        try:
            data_list = ast.literal_eval(data)  # Safely convert string to Python list/dict
            if isinstance(data_list, list):  # Extract names from the list of dictionaries
                return " ".join([item['name'] for item in data_list if 'name' in item])
        except (ValueError, SyntaxError):
            return ""
    return str(data)  # If it's not JSON-like, return as string

# Apply to genres and keywords columns
filtered_movies['genres'] = filtered_movies['genres'].fillna('[]').apply(extract_keywords_or_genres)
filtered_keywords['keywords'] = filtered_keywords['keywords'].fillna('[]').apply(extract_keywords_or_genres)


In [92]:
print(filtered_movies.head())
print(filtered_keywords.head())

                         genres    id original_language original_title  \
0       Animation Comedy Family   862                en      Toy Story   
1      Adventure Fantasy Family  8844                en        Jumanji   
5   Action Crime Drama Thriller   949                en           Heat   
9     Adventure Action Thriller   710                en      GoldenEye   
15                  Drama Crime   524                en         Casino   

   popularity release_date  vote_average  vote_count  
0   21.946943   1995-10-30           7.7      5415.0  
1   17.015539   1995-12-15           6.9      2413.0  
5   17.924927   1995-12-15           7.7      1886.0  
9   14.686036   1995-11-16           6.6      1194.0  
15  10.137389   1995-11-22           7.8      1343.0  
      id                                           keywords
0    862  jealousy toy boy friendship friends rivalry bo...
1   8844  board game disappearance based on children's b...
5    949  robbery detective bank obsession ch

In [93]:
import nltk
import string
import re
import os
from nltk.corpus import stopwords
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Custom path for Punkt tokenizer
punkt_path = os.path.join(r'C:\Users\alfem\nltk_data\tokenizers\punkt', 'english.pickle')
tokenizer = PunktSentenceTokenizer(punkt_path)

nltk.download('wordnet')  # Download the 'wordnet' resource 
nltk.download('stopwords')  # Stopwords
nltk.download('omw-1.4')  # WordNet for Lemmatization

stemmer = PorterStemmer()
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove digits
    text = re.sub(r'\d+', '', text)
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize text using the custom tokenizer
    word_tokens = tokenizer.tokenize(text)
    
    # Flatten the tokenized sentences into a single list of words
    word_tokens = [word for sentence in word_tokens for word in sentence.split()]
    
    # Remove stopwords
    tokens = [word for word in word_tokens if word not in stop_words]
    
    # Apply lemmatization
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Return the processed text as a single string
    return ' '.join(lemmatized_tokens)

# Example application (replace with your actual dataframes)
filtered_movies['genres'] = filtered_movies['genres'].apply(preprocess_text)
filtered_keywords['keywords'] = filtered_keywords['keywords'].apply(preprocess_text)


[nltk_data] Downloading package wordnet to C:\Users\alfem\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alfem\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\alfem\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [94]:
print(filtered_movies.head())
print(filtered_keywords.head())

                         genres    id original_language original_title  \
0       animation comedy family   862                en      Toy Story   
1      adventure fantasy family  8844                en        Jumanji   
5   action crime drama thriller   949                en           Heat   
9     adventure action thriller   710                en      GoldenEye   
15                  drama crime   524                en         Casino   

   popularity release_date  vote_average  vote_count  
0   21.946943   1995-10-30           7.7      5415.0  
1   17.015539   1995-12-15           6.9      2413.0  
5   17.924927   1995-12-15           7.7      1886.0  
9   14.686036   1995-11-16           6.6      1194.0  
15  10.137389   1995-11-22           7.8      1343.0  
      id                                           keywords
0    862  jealousy toy boy friendship friend rivalry boy...
1   8844  board game disappearance based childrens book ...
5    949  robbery detective bank obsession ch

In [95]:
from IPython.display import display, HTML

# Convert each DataFrame to HTML
html_movies = filtered_movies.head().to_html()
html_keywords = filtered_keywords.head().to_html()
html_ratings = filtered_ratings.head().to_html()

# Display them side by side
display(HTML(f"<div style='display: flex; gap: 20px;'>{html_movies}{html_keywords}{html_ratings}</div>"))


In [96]:
# Merge filtered_ratings with filtered_movies to include movie names
merged_data = filtered_ratings.merge(
    filtered_movies[['id', 'original_title']], 
    left_on='movieId', 
    right_on='id'
)

# Create the user-item matrix with 'userId' as rows, 'original_title' as columns, and 'rating' as values
user_item_matrix = merged_data.pivot_table(
    index='userId', 
    columns='original_title', 
    values='rating', 
    aggfunc='mean'  # Use mean in case of multiple ratings for the same movie by the same user
)

# Fill NaN values with 0 (indicating no rating)
user_item_matrix = user_item_matrix.fillna(0)



In [97]:
user_item_matrix

original_title,10 Things I Hate About You,127 Hours,13 Sins,1408,16 Blocks,2 Fast 2 Furious,20 ans d'écart,21 Grams,25th Hour,27 Dresses,...,While You Were Sleeping,White Bird in a Blizzard,Wild Things,Wild at Heart,Wizards of Waverly Place: The Movie,X-Men Origins: Wolverine,X-Men: Days of Future Past,Zathura: A Space Adventure,Zodiac,eXistenZ
userId,,,,,,,,,,,,,,,,,,,,,
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
270859,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
270879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity


similarity_matrix = cosine_similarity(user_item_matrix)

In [103]:
similarity_df = pd.DataFrame(similarity_matrix)

# Slice the DataFrame to show only the first 10 rows and columns
similarity_df = similarity_df.iloc[:10, :10]  # First 10 rows and columns

# Convert the sliced DataFrame to an HTML string
html_simmatrix = similarity_df.to_html()

# Display the HTML table
display(HTML(f"<div style='display: flex; gap: 20px;'>{html_simmatrix}</div>"))

,0,1,2,3,4,5,6,7,8,9
0,1.000000,0.413601,0.241995,0.116511,0.167361,0.275489,0.250327,0.425730,0.248315,0.243637
1,0.413601,1.000000,0.418805,0.198235,0.268903,0.315599,0.402876,0.554491,0.350133,0.517157
2,0.241995,0.418805,1.000000,0.151107,0.400443,0.438967,0.421422,0.420529,0.446321,0.338893
3,0.116511,0.198235,0.151107,1.000000,0.374406,0.350767,0.370048,0.197991,0.252060,0.250515
4,0.167361,0.268903,0.400443,0.374406,1.000000,0.433192,0.425617,0.336689,0.367616,0.324779
5,0.275489,0.315599,0.438967,0.350767,0.433192,1.000000,0.431965,0.464167,0.466021,0.265031
6,0.250327,0.402876,0.421422,0.370048,0.425617,0.431965,1.000000,0.434858,0.482612,0.379672
7,0.425730,0.554491,0.420529,0.197991,0.336689,0.464167,0.434858,1.000000,0.446048,0.472674
8,0.248315,0.350133,0.446321,0.252060,0.367616,0.466021,0.482612,0.446048,1.000000,0.364530
9,0.243637,0.517157,0.338893,0.250515,0.324779,0.265031,0.379672,0.472674,0.364530,1.000000


In [ ]:
select_userid =